# API SPOTIFY

In [1]:
pip install spotipy

   ---------------------------------------- 0.0/261.5 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/261.5 kB 660.6 kB/s eta 0:00:01
   --------------- ------------------------ 102.4/261.5 kB 1.2 MB/s eta 0:00:01
   ------------------------------------ --- 235.5/261.5 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 261.5/261.5 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials

client_id = "748f4cce341d436cb9ae583633bdafd6"
client_secret = "bdcc7c65a9244123a26d9472d3dc6865"
redirect_uri="http://localhost:8888/callback",
scope="playlist-read-private"

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [4]:
import requests

CLIENT_ID = "748f4cce341d436cb9ae583633bdafd6"
CLIENT_SECRET = "bdcc7c65a9244123a26d9472d3dc6865"

url = "https://accounts.spotify.com/api/token"
payload = {
    "grant_type": "client_credentials",
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET
}

response = requests.post(url, data=payload)
access_token = response.json().get("access_token")

print("Access Token:", access_token)


Access Token: BQA9d9MQOoJcnxvS1yDSo2MIiteCGoUeGBRli8zGSUaEzRK8E7R4RkKL0JHeNV_w0BvCSSl2EJbiEKzP_oGi8ReKprAFQJcCWeRTTDGx5YoE-tJqiAvjJ_laR15S7G92Ekc6wND8inA


Docs API = https://spotipy.readthedocs.io/en/2.22.1/#spotipy.client.Spotify.category_playlists

In [5]:
import requests
import pandas as pd

# 👉 Reemplaza esto con tu Access Token válido
ACCESS_TOKEN1 = access_token

# Función para obtener canciones populares por año
def obtener_canciones_por_año(año):
    url = "https://api.spotify.com/v1/search"
    params = {
        "q": f"year:{año}",
        "type": "track",
        "limit": 50  # Puedes cambiar este valor para obtener más resultados
    }
    headers = {"Authorization": f"Bearer {ACCESS_TOKEN1}"}

    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    canciones = []
    if "tracks" in data:
        for item in data["tracks"]["items"]:
            canciones.append({
                "nombre": item["name"],
                "artista": item["artists"][0]["name"],
                "id_artista": item["artists"][0]["id"],
                "año": año
            })
    return canciones

# Obtener datos para varios años
años = list(range(2010, 2026))  # Desde 2010 hasta 2025
canciones_por_año = []

for año in años:
    canciones_por_año.extend(obtener_canciones_por_año(año))

# Convertir a DataFrame
df_canciones = pd.DataFrame(canciones_por_año)
print(df_canciones.head())

# Guardar en CSV
df_canciones.to_csv("canciones_por_año.csv", index=False)


                         nombre       artista              id_artista   año
0          Just the Way You Are    Bruno Mars  0du5cEVh5yTK9QJze8zA0C  2010
1                    Angels Cry  Mariah Carey  4iHNK0tOyZPYnBU7nGAgpQ  2010
2      Only Girl (In The World)       Rihanna  5pKCCKE2ajJHZ9KAiaK11H  2010
3  The Real Slim Shady - Edited        Eminem  7dGJo4pcD2V6oG8kP0tJRR  2010
4                     The Spins    Mac Miller  4LLpKhyESsyAXpc4laK94U  2010


In [6]:
# Función para obtener el género del artista
def obtener_genero_artista(artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}"
    headers = {"Authorization": f"Bearer {ACCESS_TOKEN1}"}

    response = requests.get(url, headers=headers)
    data = response.json()

    if "genres" in data and data["genres"]:
        return data["genres"][0]  # Tomamos el primer género
    return "Desconocido"

# Aplicar la función a todas las canciones
df_canciones["genero"] = df_canciones["id_artista"].apply(obtener_genero_artista)

# Guardar con géneros
df_canciones.to_csv("canciones_con_genero.csv", index=False)

print(df_canciones.head())


                         nombre       artista              id_artista   año  \
0          Just the Way You Are    Bruno Mars  0du5cEVh5yTK9QJze8zA0C  2010   
1                    Angels Cry  Mariah Carey  4iHNK0tOyZPYnBU7nGAgpQ  2010   
2      Only Girl (In The World)       Rihanna  5pKCCKE2ajJHZ9KAiaK11H  2010   
3  The Real Slim Shady - Edited        Eminem  7dGJo4pcD2V6oG8kP0tJRR  2010   
4                     The Spins    Mac Miller  4LLpKhyESsyAXpc4laK94U  2010   

        genero  
0  Desconocido  
1    christmas  
2  Desconocido  
3          rap  
4  Desconocido  


In [19]:
import pandas as pd
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Configurar credenciales de la API de Spotify (sustituye con tus claves)
SPOTIFY_CLIENT_ID = "748f4cce341d436cb9ae583633bdafd6"
SPOTIFY_CLIENT_SECRET = "bdcc7c65a9244123a26d9472d3dc6865"

# Autenticación con Spotify
auth_manager = SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

def obtener_genero(artista):
    try:
        print(f"🔍 Buscando género para: {artista}")  # Debug

        artista = artista.strip()
        resultados = sp.search(q=f"artist:{artista}", type="artist", limit=1)

        artistas = resultados.get("artists", {}).get("items", [])

        if artistas:
            artista_spotify = artistas[0]  # Primer resultado
            nombre_real = artista_spotify.get("name", "Desconocido")
            id_artista = artista_spotify.get("id", "Sin ID")
            generos = artista_spotify.get("genres", [])

            print(f"✅ Encontrado: {nombre_real} ({id_artista}) → Géneros: {generos}")

            return generos[0] if generos else "Sin datos"

        else:
            print(f"⚠ No se encontró el artista: {artista}")
            return "No encontrado"

    except Exception as e:
        print(f"❌ Error con {artista}: {e}")
        return "Error"

print(obtener_genero("Rihanna"))



🔍 Buscando género para: Rihanna
✅ Encontrado: Rihanna (5pKCCKE2ajJHZ9KAiaK11H) → Géneros: []
Sin datos


In [7]:
df_generos1 = pd.read_csv("C:\\Users\\Rober\\Desktop\\PROYECTS\\Music Trends Proyect\\CSV\\canciones_con_genero.csv")
df_generos1

,nombre,artista,id_artista,año,genero
0,Just the Way You Are,Bruno Mars,0du5cEVh5yTK9QJze8zA0C,2010,Desconocido
1,Angels Cry,Mariah Carey,4iHNK0tOyZPYnBU7nGAgpQ,2010,christmas
2,Only Girl (In The World),Rihanna,5pKCCKE2ajJHZ9KAiaK11H,2010,Desconocido
3,The Real Slim Shady - Edited,Eminem,7dGJo4pcD2V6oG8kP0tJRR,2010,rap
4,The Spins,Mac Miller,4LLpKhyESsyAXpc4laK94U,2010,Desconocido
...,...,...,...,...,...
793,Are U Still - Live,Leven Kali,5YZ5AExR68U3ZblH6HcO6B,2025,Desconocido
794,Greenlight,Tate McRae,45dkTj5sMRSjrmBSBeiHym,2025,Desconocido
795,Scratch My Brain (Interlude) (Visualizer),Elohim,6wKxOKEA3K6R2UZ3COLXEY,2025,Desconocido
796,Means I care,Tate McRae,45dkTj5sMRSjrmBSBeiHym,2025,Desconocido


In [9]:
# Contar la frecuencia de cada género por año
df_frecuencia = df_canciones.groupby(["año", "genero"]).size().reset_index(name="frecuencia")

# Guardar los datos
df_frecuencia.to_csv("generos_populares_por_año.csv", index=False)

# Mostrar los 10 géneros más frecuentes por año
print(df_frecuencia.sort_values(["año", "frecuencia"], ascending=[True, False]).groupby("año").head(10))


      año          genero  frecuencia
0    2010     Desconocido          23
16   2010             rap           6
14   2010             pop           3
1    2010       christmas           2
3    2010             edm           2
..    ...             ...         ...
279  2025  brooklyn drill           1
280  2025   chamber music           1
281  2025  christian folk           1
282  2025         country           1
283  2025         doo-wop           1

[160 rows x 3 columns]


In [39]:
# Diccionario para agrupar géneros en categorías principales
mapa_generos = {
    "pop": ["pop", "dance pop", "christmas pop", "electropop","christmas","soft pop","indonesian pop","nederpop","art pop","schlager","baroque pop","cantopop","mandopop","britpop","german pop","french pop","schlagerparty","chamber pop","brazilian pop","norwegian pop","bedroom pop","iskelmä","psychedelic pop","city pop","pop urbaine","gujarati pop"],
    "rock": ["rock", "alternative rock", "hard rock","folk rock","psychedelic rock","rock en español","post-rock","grunge","modern rock","pop punk","emo","classic rock","neue deutsche welle","pinoy alternative","new wave","brazilian rock","thai rock","post-punk","shoegaze"],
    "hip-hop": ["hip hop", "rap", "trap", "gangsta rap","experimental hip hop","german hip hop","east coast hip hop","brazilian hip hop","melodic rap","emo rap","italian trap","rage rap","egyptian hip hop","grime","argentine trap","trap latino","boom bap","uk drill","brooklyn drill"],
    "electrónica": ["edm", "house", "techno", "trance", "drum and bass","jungle","minimal techno","tropical house","future bass","electronica","tech house","cedm","idm","french house","electroclash"],
    "latino": ["reggaeton", "latin pop", "salsa", "bachata", "tropical","latin alternative","bolero","cumbia","champeta","reggaeton chileno","rkt"],
    "metal": ["metal", "death metal", "black metal", "metalcore","symphonic metal","nu metal","melodic hardcore","slowcore","horrorcore","nightcore"],
    "jazz": ["jazz", "smooth jazz", "cool jazz","french jazz","doo-wop"],
    "clásica": ["classical", "baroque", "orchestral","opera","neoclassical","choral","chamber music"],
    "folk": ["folk", "indie folk", "americana", "folk pop","phleng phuea chiwit","christian folk"],
    "blues": ["blues", "delta blues", "chicago blues","southern gospel","retro soul","soul","gospel","northern soul"],
    "country": ["country", "modern country", "classic country","bluegrass"],
    "reggae": ["reggae", "dancehall", "roots reggae","spanish-language reggae"],
    "r&b": ["r&b","alternative r&b"],
    "samba" : ["pagode", "samba", "axé","mpb","pagode baiano","sertanejo","sertanejo tradicional","forró","nova mpb","agronejo"],
    "africano" : ["afrobeat", "afropop","kizomba","afrobeats"],
    "k-pop" : ["k-pop", "k-hip hop","thai pop","c-pop","t-pop"],
    "indie" : ["indie pop", "indie folk", "indie rock","indie","piano house","tamil indie"],
    "bollywood" : ["bollywood","bangla pop","mollywood"],
    "dance" : ["post-disco","italo disco"],
    "flamenco" : ["flamenco","flamenco pop"]

}


In [40]:
# Función para asignar género general
def asignar_genero_general(genero_especifico):
    for categoria, lista_generos in mapa_generos.items():
        if genero_especifico in lista_generos:
            return categoria
    return "Otros"  # Si no encontramos coincidencia, lo ponemos en "Otros"

# Aplicar la función a la columna de géneros
df_canciones["genero_general"] = df_canciones["genero"].apply(asignar_genero_general)

# Guardar en CSV
df_canciones.to_csv("canciones_con_genero_general.csv", index=False)
print(df_canciones[["nombre", "genero", "genero_general"]].head(20))


                                       nombre         genero genero_general
0                        Just the Way You Are    Desconocido          Otros
1                                  Angels Cry      christmas            pop
2                    Only Girl (In The World)    Desconocido          Otros
3                The Real Slim Shady - Edited            rap        hip-hop
4                                   The Spins    Desconocido          Otros
5                             Smooth Criminal    Desconocido          Otros
6                                         S&M    Desconocido          Otros
7              Estuve a punto de - Entrevista      latin pop         latino
8   DJ Got Us Fallin' In Love (feat. Pitbull)            r&b            r&b
9                                       Alice    Desconocido          Otros
10                                  Marry You    Desconocido          Otros
11                                 Let You Go  drum and bass    electrónica
12          

In [41]:
# Contar la frecuencia de cada género general por año
df_frecuencia = df_canciones.groupby(["año", "genero_general"]).size().reset_index(name="frecuencia")

# Guardar los datos
df_frecuencia.to_csv("generos_mas_populares_por_año.csv", index=False)

# Mostrar los 5 géneros más populares por año
print(df_frecuencia.sort_values(["año", "frecuencia"], ascending=[True, False]).groupby("año").head(5))


      año genero_general  frecuencia
0    2010          Otros          23
6    2010            pop           8
2    2010        hip-hop           6
1    2010    electrónica           3
9    2010          samba           3
..    ...            ...         ...
157  2025          Otros          35
162  2025        hip-hop           3
160  2025        country           2
166  2025            pop           2
158  2025          blues           1

[80 rows x 3 columns]


In [33]:
# Revisar los géneros que quedaron en "Otros"
generos_otros = df_canciones[df_canciones["genero_general"] == "Otros"]["genero"].unique()
print(generos_otros)


['Desconocido' 'anime' 'lullaby' 'oyun havasi' 'celtic'
 'singer-songwriter' 'comedy' "children's music" 'christian' 'avant-garde'
 'medieval' 'devotional' 'sholawat' 'italian singer-songwriter' 'opm'
 'alternative r&b']
